🤖 AI God Multimodal 5-in-1 - Google Colab
Notebook นี้รวมโมเดลหลักไว้ 5 แบบ:

🧠 Mistral 7B (Text Generation)
🧠 Phi-2 (เบาเร็ว)
🧠 TinyLlama (เร็วมาก / สำรอง)
🎤 Whisper (เสียงพูดเป็นข้อความ)
🎨 Stable Diffusion (วาดภาพจากข้อความ)
✅ มีระบบ UI ด้วย Gradio + ปุ่มพิเศษ + รองรับเสียงพูด + การเลือกโมเดล

In [ ]:
# 📦 STEP 1: ติดตั้งไลบรารีที่จำเป็น
# แก้ไขเวอร์ชัน Gradio เพื่อให้แสดงลิงก์สาธารณะได้
# แก้ไขเวอร์ชันของ numpy และ websockets เพื่อแก้ปัญหา dependency conflicts
!pip install -q llama-cpp-python==0.2.64 gradio==3.48.0 transformers accelerate torchaudio diffusers openai-whisper numpy==1.24.4 websockets==11.0.3
!apt install -y ffmpeg

In [ ]:
# 📁 STEP 2: ดาวน์โหลดโมเดล (เลือกเฉพาะที่ใช้)
# สำคัญ: ต้องดาวน์โหลดไฟล์ .gguf ไปยังโฟลเดอร์ 'models'
import os
from huggingface_hub import hf_hub_download

HF_MODELS = {
    'mistral': {
        'repo_id': 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF',
        'filename': 'mistral-7b-instruct-v0.1.Q4_K_M.gguf'
    },
    'phi2': {
        'repo_id': 'TheBloke/phi-2-GGUF',
        'filename': 'phi-2.Q4_K_M.gguf'
    },
    'tinyllama': {
        'repo_id': 'TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF',
        'filename': 'tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf' # แก้ไขชื่อไฟล์ TinyLlama
    }
}
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)

print("Downloading models... This may take some time.")

# ดาวน์โหลด Mistral (เป็นตัวอย่าง คุณสามารถเลือกดาวน์โหลดตัวอื่นหรือทั้งหมดได้)
# หากคุณต้องการใช้ Phi-2 หรือ TinyLlama ต้องแก้บรรทัดนี้หรือเพิ่มบรรทัดอื่นเข้ามา
# แนะนำให้ดาวน์โหลดแค่โมเดลที่คุณจะใช้จริงๆ เพราะไฟล์มีขนาดใหญ่มาก
# Mistral 7B เป็นตัวเลือกที่ดีสำหรับการเริ่มต้น
hf_hub_download(
    repo_id=HF_MODELS['mistral']['repo_id'],
    filename=HF_MODELS['mistral']['filename'],
    local_dir=MODEL_DIR,
    local_dir_use_symlinks=False
)

# หากต้องการดาวน์โหลด Phi-2 ด้วย ให้เปิดคอมเมนต์บรรทัดนี้
# hf_hub_download(
#     repo_id=HF_MODELS['phi2']['repo_id'],
#     filename=HF_MODELS['phi2']['filename'],
#     local_dir=MODEL_DIR,
#     local_dir_use_symlinks=False
# )

# หากต้องการดาวน์โหลด TinyLlama ด้วย ให้เปิดคอมเมนต์บรรทัดนี้
# hf_hub_download(
#     repo_id=HF_MODELS['tinyllama']['repo_id'],
#     filename=HF_MODELS['tinyllama']['filename'],
#     local_dir=MODEL_DIR,
#     local_dir_use_symlinks=False
# )

print("Model download complete.")

In [ ]:
# 🧠 STEP 3: โหลดโมเดล LLM
from llama_cpp import Llama

# ตรวจสอบว่าไฟล์โมเดลที่เราจะโหลดมีอยู่จริงหรือไม่
mistral_model_path = f"{MODEL_DIR}/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
# phi2_model_path = f"{MODEL_DIR}/phi-2.Q4_K_M.gguf" # เปิดคอมเมนต์หากดาวน์โหลด Phi-2
# tinyllama_model_path = f"{MODEL_DIR}/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf" # เปิดคอมเมนต์หากดาวน์โหลด TinyLlama

# โหลด Mistral (ต้องแน่ใจว่าได้ดาวน์โหลดไฟล์นี้ใน STEP 2 แล้ว)
try:
    mistral = Llama(model_path=mistral_model_path, n_ctx=2048, n_gpu_layers=-1, verbose=False) # n_gpu_layers=-1 ใช้ GPU ทั้งหมด
    print("Mistral model loaded.")
except FileNotFoundError:
    print(f"Mistral model not found at {mistral_model_path}. Please ensure it was downloaded in STEP 2.")
    mistral = None # ตั้งค่าเป็น None เพื่อป้องกันข้อผิดพลาดหากโมเดลไม่ถูกโหลด

# โหลด Phi-2 (เปิดคอมเมนต์หากต้องการใช้ และต้องแน่ใจว่าดาวน์โหลดใน STEP 2 แล้ว)
# try:
#     phi2 = Llama(model_path=phi2_model_path, n_ctx=2048, n_gpu_layers=-1, verbose=False)
#     print("Phi-2 model loaded.")
# except FileNotFoundError:
#     print(f"Phi-2 model not found at {phi2_model_path}. Please ensure it was downloaded in STEP 2.")
#     phi2 = None

# โหลด TinyLlama (เปิดคอมเมนต์หากต้องการใช้ และต้องแน่ใจว่าดาวน์โหลดใน STEP 2 แล้ว)
# try:
#     tinyllama = Llama(model_path=tinyllama_model_path, n_ctx=1024, n_gpu_layers=-1, verbose=False)
#     print("TinyLlama model loaded.")
# except FileNotFoundError:
#     print(f"TinyLlama model not found at {tinyllama_model_path}. Please ensure it was downloaded in STEP 2.")
#     tinyllama = None


def choose_model(name):
    # ปรับปรุงฟังก์ชัน choose_model ให้คืนค่าเฉพาะโมเดลที่โหลดสำเร็จ
    if name == 'Mistral' and ('mistral' in locals() and mistral):
        return mistral
    elif name == 'Phi-2' and ('phi2' in locals() and phi2): # เปิดคอมเมนต์หากใช้ Phi-2
        return phi2
    elif name == 'TinyLlama' and ('tinyllama' in locals() and tinyllama): # เปิดคอมเมนต์หากใช้ TinyLlama
        return tinyllama
    else:
        print(f"Warning: Model '{name}' not loaded or not available. Defaulting to Mistral (if available).")
        return mistral if ('mistral' in locals() and mistral) else None # คืนค่า Mistral หากมี หรือ None หากไม่มีอะไรเลย

In [ ]:
# 🎤 STEP 4: ฟังก์ชันแปลงเสียงพูดเป็นข้อความ
import whisper
asr = whisper.load_model("base") # ใช้โมเดลขนาด "base" หรือ "small" เพื่อความเร็ว

def transcribe(audio):
    if audio is None:
        return "No audio provided."
    try:
        # Colab บางครั้งอาจบันทึกไฟล์ .webm, whisper ต้องการ .flac, .wav, .mp3, etc.
        # ตรวจสอบรูปแบบไฟล์ก่อนส่งให้ whisper หรือแปลงให้ถูกต้อง
        # ในกรณีส่วนใหญ่ whisper สามารถจัดการไฟล์ที่ Gradio ส่งมาได้โดยตรง
        return asr.transcribe(audio)["text"]
    except Exception as e:
        return f"Error transcribing audio: {e}"

In [ ]:
# 🎨 STEP 5: ฟังก์ชันวาดภาพจากข้อความ
from diffusers import StableDiffusionPipeline
import torch

# ตรวจสอบว่ามี GPU หรือไม่ และใช้การตั้งค่าที่เหมาะสม
if torch.cuda.is_available():
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
    pipe.to("cuda")
    print("Stable Diffusion loaded on CUDA.")
else:
    print("CUDA not available. Stable Diffusion will run on CPU, which might be very slow.")
    pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4") # ไม่ใช้ float16 บน CPU
    pipe.to("cpu")

def generate_image(prompt):
    if not prompt:
        return None # ไม่สร้างภาพถ้าไม่มี prompt
    try:
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

In [ ]:
# 🧠🖼️🎤 STEP 6: Gradio UI
import gradio as gr

def chat(text, model_choice):
    if not text:
        return "กรุณาพิมพ์คำถามของคุณ"

    llm = choose_model(model_choice)
    if llm is None:
        return f"โมเดล {model_choice} ไม่พร้อมใช้งาน กรุณาตรวจสอบการดาวน์โหลดและการโหลดโมเดล"

    try:
        # Llama.cpp prompt format สำหรับ Mistral Instruct
        # อาจต้องปรับตามโมเดล LLM ที่เลือกใช้
        if model_choice == "Mistral":
            formatted_prompt = f"<s>[INST] {text} [/INST]"
        else: # สำหรับ Phi-2, TinyLlama หรือโมเดลอื่นๆ
            formatted_prompt = text

        result = llm(formatted_prompt, max_tokens=500)['choices'][0]['text']
        return result.strip()
    except Exception as e:
        return f"เกิดข้อผิดพลาดในการสนทนากับโมเดล: {e}"

with gr.Blocks() as demo:
    gr.Markdown("## 🔮 AI God - Multimodal 5-in-1")
    with gr.Row():
        # แสดงเฉพาะโมเดลที่โหลดสำเร็จ
        available_models = []
        if 'mistral' in locals() and mistral:
            available_models.append("Mistral")
        if 'phi2' in locals() and phi2:
            available_models.append("Phi-2")
        if 'tinyllama' in locals() and tinyllama:
            available_models.append("TinyLlama")

        if not available_models:
            gr.Markdown("**<span style='color:red'>! ไม่มีโมเดล LLM ใดๆ โหลดสำเร็จ โปรดตรวจสอบ STEP 2 และ STEP 3. !</span>**")
            model_choice = gr.Dropdown(choices=[], value=None, label="🧠 เลือกโมเดล (ไม่มีโมเดลพร้อมใช้งาน)", interactive=False)
        else:
            model_choice = gr.Dropdown(choices=available_models, value=available_models[0], label="🧠 เลือกโมเดล")

    with gr.Row():
        txt = gr.Textbox(label="💬 พิมพ์คำถาม/คำสั่ง")
        btn = gr.Button("🪄 ปลดปล่อยพลังงาน")
        out = gr.Textbox(label="📤 คำตอบ", lines=5) # เพิ่ม lines ให้แสดงผลได้เยอะขึ้น
    btn.click(fn=chat, inputs=[txt, model_choice], outputs=out)

    gr.Markdown("___") # เพิ่มเส้นแบ่ง
    gr.Markdown("### 🎤 Whisper (พูดแล้วแปลง)")
    mic = gr.Audio(source="microphone", type="filepath", label="กดเพื่อพูด")
    txt_out = gr.Textbox(label="ข้อความเสียงที่แปลงแล้ว")
    mic.change(transcribe, inputs=mic, outputs=txt_out)

    gr.Markdown("___") # เพิ่มเส้นแบ่ง
    gr.Markdown("### 🎨 วาดภาพจากข้อความ")
    prompt_img = gr.Textbox(label="Prompt สำหรับสร้างภาพ (ภาษาอังกฤษ)", placeholder="A cat in space, digital art") # เปลี่ยนชื่อตัวแปรเพื่อไม่ให้ซ้ำ
    btn_img = gr.Button("🖼️ สร้างภาพ") # เพิ่มปุ่มสำหรับสร้างภาพ
    img_out = gr.Image(label="ภาพที่สร้าง")
    btn_img.click(generate_image, inputs=prompt_img, outputs=img_out)


print("Launching Gradio UI...")
try:
    demo.launch(share=True)
except Exception as e:
    print(f"Failed to launch Gradio UI: {e}")
    print("This might be due to network issues or resource limitations in Colab.")